In [1]:
import json
import os
import time
import random
from PIL import Image
import numpy as np
import matplotlib.pyplot as plt
from tqdm import tqdm
import shutil

import torch
import torch.nn as nn
import torch.backends.cudnn as cudnn
import torch.optim as optim
from torch.utils.data import DataLoader
from torchvision import datasets
from torchvision import transforms
from torchvision import models
from torchsummary import summary
from sklearn.model_selection import train_test_split

from model_pytorch import EfficientNet
from utils import Bar,Logger, AverageMeter, accuracy, mkdir_p, savefig
from warmup_scheduler import GradualWarmupScheduler

from PIL import ImageFile
ImageFile.LOAD_TRUNCATED_IMAGES = True
import collections

In [2]:
# GPU Device
gpu_id = 3
os.environ['CUDA_VISIBLE_DEVICES'] = str(gpu_id)
use_cuda = torch.cuda.is_available()
print("GPU device %d:" %(gpu_id), use_cuda)

GPU device 3: True


# Arguments

In [3]:
source_dir = '/media/data2/dataset/GAN_ImageData/StyleGAN_256/'
target_dir = '/media/data2/dataset/GAN_ImageData/PGGAN_128/'

In [4]:
pretrained = './log/style1/128/b0/checkpoint.pth.tar'
resume = ''

In [6]:
# Model
model_name = 'efficientnet-b0' # b0-b7 scale

# Optimization
num_classes = 2
epochs = 5000
start_epoch = 0
train_batch = 32
test_batch = 300
lr = 0.01
schedule = [1500, 3000]
momentum = 0
gamma = 0.1 # LR is multiplied by gamma on schedule

# CheckPoint
checkpoint = './log/style1/128/b0/to_pggan/l2sp_style1' # dir
if not os.path.isdir(checkpoint):
    os.mkdir(checkpoint)
num_workers = 4

# Seed
manual_seed = 7
random.seed(manual_seed)
torch.cuda.manual_seed_all(manual_seed)

# Image
size = (128, 128)

# sp
alpha = 0.1
beta = 0.1
fc_name = 'fc.'

best_acc = 0

In [7]:
state = {}
state['num_classes'] = num_classes
state['epochs'] = epochs
state['start_epoch'] = start_epoch
state['train_batch'] = train_batch
state['test_batch'] = test_batch
state['lr'] = lr
state['schedule'] = schedule
state['momentum'] = momentum
state['gamma'] = gamma

# Dataset

In [8]:
train_dir = os.path.join(target_dir, '100_shot_style1')
val_target_dir = os.path.join(target_dir, 'validation')
val_source_dir = os.path.join(source_dir, 'validation')

train_aug = transforms.Compose([
    transforms.RandomAffine(degrees=2, translate=(0.02, 0.02), scale=(0.98, 1.02), shear=2, fillcolor=(124,117,104)),
    transforms.Resize(size),
    transforms.RandomHorizontalFlip(),
    transforms.ToTensor(),
    transforms.RandomErasing(p=0.3, scale=(0.02, 0.10), ratio=(0.3, 3.3), value=0, inplace=True),
])
val_aug = transforms.Compose([
    transforms.Resize(size),
    transforms.ToTensor(),
])

# pin_memory : cuda pin memeory use
train_loader = DataLoader(datasets.ImageFolder(train_dir, transform=train_aug),
                          batch_size=train_batch, shuffle=True, num_workers=num_workers, pin_memory=True)
val_target_loader = DataLoader(datasets.ImageFolder(val_target_dir, val_aug),
                       batch_size=test_batch, shuffle=True, num_workers=num_workers, pin_memory=True)
val_source_loader = DataLoader(datasets.ImageFolder(val_source_dir, val_aug),
                       batch_size=test_batch, shuffle=True, num_workers=num_workers, pin_memory=True)

# Model

In [9]:
source_model = EfficientNet.from_name(model_name, num_classes=num_classes, override_params={'dropout_rate':0.0})
model = EfficientNet.from_name(model_name, num_classes=num_classes, override_params={'dropout_rate':0.0})

# Pre-trained
if pretrained:
    print("=> using pre-trained model '{}'".format(pretrained))
    source_model.load_state_dict(torch.load(pretrained)['state_dict'])
    model.load_state_dict(torch.load(pretrained)['state_dict'])

=> using pre-trained model './log/style1/128/b0/checkpoint.pth.tar'


In [10]:
source_model.to('cuda')
model.to('cuda')
cudnn.benchmark = True
print('    Total params: %.2fM' % (sum(p.numel() for p in model.parameters())/1000000.0))

    Total params: 4.01M


In [11]:
for param in source_model.parameters():
    param.requires_grad = False
source_model.eval()

EfficientNet(
  (_conv_stem): Conv2dStaticSamePadding(
    3, 32, kernel_size=(3, 3), stride=(2, 2), bias=False
    (static_padding): ZeroPad2d(padding=(0, 1, 0, 1), value=0.0)
  )
  (_bn0): GroupNorm(8, 32, eps=1e-05, affine=True)
  (_blocks): ModuleList(
    (0): MBConvBlock(
      (_depthwise_conv): Conv2dStaticSamePadding(
        32, 32, kernel_size=(3, 3), stride=[1, 1], groups=32, bias=False
        (static_padding): ZeroPad2d(padding=(1, 1, 1, 1), value=0.0)
      )
      (_bn1): GroupNorm(8, 32, eps=1e-05, affine=True)
      (_se_reduce): Conv2dStaticSamePadding(
        32, 8, kernel_size=(1, 1), stride=(1, 1)
        (static_padding): Identity()
      )
      (_se_expand): Conv2dStaticSamePadding(
        8, 32, kernel_size=(1, 1), stride=(1, 1)
        (static_padding): Identity()
      )
      (_project_conv): Conv2dStaticSamePadding(
        32, 16, kernel_size=(1, 1), stride=(1, 1), bias=False
        (static_padding): Identity()
      )
      (_bn2): GroupNorm(4, 16, ep

In [12]:
source_model_weights = {}
for name, param in source_model.named_parameters():
    source_model_weights[name] = param.detach()

# Model tuning

In [13]:
def reg_cls(model):
    l2_cls = torch.tensor(0.).cuda()
    for name, param in model.named_parameters():
        if name.startswith(fc_name):
            l2_cls += 0.5 * torch.norm(param) ** 2
    return l2_cls

def reg_l2sp(model):
    sp_loss = torch.tensor(0.).cuda()
    for name, param in model.named_parameters():
        if not name.startswith(fc_name):
            sp_loss += 0.5 * torch.norm(param - source_model_weights[name]) ** 2
    return sp_loss

In [14]:
summary(model, input_size=(3,128,128),device='cuda')

----------------------------------------------------------------
        Layer (type)               Output Shape         Param #
         ZeroPad2d-1          [-1, 3, 129, 129]               0
Conv2dStaticSamePadding-2           [-1, 32, 64, 64]             864
         GroupNorm-3           [-1, 32, 64, 64]              64
MemoryEfficientSwish-4           [-1, 32, 64, 64]               0
         ZeroPad2d-5           [-1, 32, 66, 66]               0
Conv2dStaticSamePadding-6           [-1, 32, 64, 64]             288
         GroupNorm-7           [-1, 32, 64, 64]              64
MemoryEfficientSwish-8           [-1, 32, 64, 64]               0
          Identity-9             [-1, 32, 1, 1]               0
Conv2dStaticSamePadding-10              [-1, 8, 1, 1]             264
MemoryEfficientSwish-11              [-1, 8, 1, 1]               0
         Identity-12              [-1, 8, 1, 1]               0
Conv2dStaticSamePadding-13             [-1, 32, 1, 1]             288
         I

# Loss

In [15]:
criterion = nn.CrossEntropyLoss().cuda()
optimizer = optim.SGD(filter(lambda p: p.requires_grad, model.parameters()), lr=lr, momentum=momentum)
# optimizer = optim.Adam(model.parameters(), weight_decay=0)
scheduler_cosine = torch.optim.lr_scheduler.CosineAnnealingLR(optimizer, epochs)
scheduler_warmup = GradualWarmupScheduler(optimizer, multiplier=8, total_epoch=10, after_scheduler=scheduler_cosine)

In [16]:
# Resume
if resume:
    print('==> Resuming from checkpoint..')
    checkpoint = os.path.dirname(resume)
#     checkpoint = torch.load(resume)
    resume = torch.load(resume)
    best_acc = resume['best_acc']
    start_epoch = resume['epoch']
    model.load_state_dict(resume['state_dict'])
    optimizer.load_state_dict(resume['optimizer'])
    logger = Logger(os.path.join(checkpoint, 'log.txt'), resume=True)
else:
    logger = Logger(os.path.join(checkpoint, 'log.txt'))
    logger.set_names(['Learning Rate', 'Train Loss', 'Valid Loss', 'Train Acc.', 'Valid Acc.', 'Source Loss', 'Source ACC.'])

# Train

In [17]:
def train(train_loader, model, criterion, optimizer, epoch, use_cuda):
    model.train()
    torch.set_grad_enabled(True)
    
    batch_time = AverageMeter()
    data_time = AverageMeter()
    losses = AverageMeter()
    top1 = AverageMeter()
    end = time.time()
    
    bar = Bar('Processing', max=len(train_loader))
    for batch_idx, (inputs, targets) in enumerate(train_loader):
        batch_size = inputs.size(0)
        if batch_size < train_batch:
            continue
        # measure data loading time
        data_time.update(time.time() - end)

        if use_cuda:
            inputs, targets = inputs.cuda(), targets.cuda()
        inputs, targets = torch.autograd.Variable(inputs), torch.autograd.Variable(targets)

        # compute output
        outputs = model(inputs)
        loss_main = criterion(outputs, targets)
        loss_cls = 0
        loss_sp = 0
        loss = loss_main + alpha*loss_sp + beta*loss_cls

        # measure accuracy and record loss
        prec1 = accuracy(outputs.data, targets.data)
        losses.update(loss.data.tolist(), inputs.size(0))
        top1.update(prec1[0], inputs.size(0))

        # compute gradient and do SGD step
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()

        # measure elapsed time
        batch_time.update(time.time() - end)
        end = time.time()

        # plot progress
        bar.suffix  = '({batch}/{size}) Data: {data:.3f}s | Batch: {bt:.3f}s | Total: {total:} | ETA: {eta:} | Loss: {loss:.4f} | top1: {top1: .4f} '.format(
                    batch=batch_idx + 1,
                    size=len(train_loader),
                    data=data_time.val,
                    bt=batch_time.val,
                    total=bar.elapsed_td,
                    eta=bar.eta_td,
                    loss=losses.avg,
                    top1=top1.avg,
                    )
        bar.next()
        if batch_idx % 10 == 0:
            print('{batch}/{size} | Total:{total:} | ETA:{eta:} | Loss:{loss:} | top1:{tp1:}'.format(
                 batch=batch_idx+1, size=len(train_loader), total=bar.elapsed_td, eta=bar.eta_td, loss=losses.avg, tp1=top1.avg))
    print('{batch}/{size} | Total:{total:} | ETA:{eta:} | Loss:{loss:} | top1:{tp1:}'.format(
                 batch=batch_idx+1, size=len(train_loader), total=bar.elapsed_td, eta=bar.eta_td, loss=losses.avg, tp1=top1.avg))
    bar.finish()
    return (losses.avg, top1.avg)

In [18]:
def test(val_loader, model, criterion, epoch, use_cuda):
    global best_acc

    batch_time = AverageMeter()
    data_time = AverageMeter()
    losses = AverageMeter()
    top1 = AverageMeter()

    # switch to evaluate mode
    model.eval()
    torch.set_grad_enabled(False)

    end = time.time()
    bar = Bar('Processing', max=len(val_loader))
    for batch_idx, (inputs, targets) in enumerate(val_loader):
        # measure data loading time
        data_time.update(time.time() - end)

        if use_cuda:
            inputs, targets = inputs.cuda(), targets.cuda()
        inputs, targets = torch.autograd.Variable(inputs, volatile=True), torch.autograd.Variable(targets)

        # compute output
        outputs = model(inputs)
        loss_main = criterion(outputs, targets)
        loss_cls = 0
        loss_sp = 0
        loss = loss_main + alpha*loss_sp + beta*loss_cls

        # measure accuracy and record loss
        prec1 = accuracy(outputs.data, targets.data)
        losses.update(loss.data.tolist(), inputs.size(0))
        top1.update(prec1[0], inputs.size(0))

        # measure elapsed time
        batch_time.update(time.time() - end)
        end = time.time()

        # plot progress
        bar.suffix  = '({batch}/{size}) Data: {data:.3f}s | Batch: {bt:.3f}s | Total: {total:} | ETA: {eta:} | Loss: {loss:} | top1: {top1:}'.format(
                    batch=batch_idx + 1,
                    size=len(val_loader),
                    data=data_time.avg,
                    bt=batch_time.avg,
                    total=bar.elapsed_td,
                    eta=bar.eta_td,
                    loss=losses.avg,
                    top1=top1.avg,)
        bar.next()
    print('{batch}/{size} | Total:{total:} | ETA:{eta:} | Loss:{loss:} | top1:{tp1:}'.format(
         batch=batch_idx+1, size=len(val_loader), total=bar.elapsed_td, eta=bar.eta_td, loss=losses.avg, tp1=top1.avg))
    bar.finish()
    return (losses.avg, top1.avg)

In [19]:
def save_checkpoint(state, is_best, checkpoint='checkpoint', filename='checkpoint.pth.tar'):
    filepath = os.path.join(checkpoint, filename)
    torch.save(state, filepath)
    if is_best:
        shutil.copyfile(filepath, os.path.join(checkpoint, 'model_best.pth.tar'))

def adjust_learning_rate(optimizer, epoch):
    global state
    lr_set = [1, 1e-1, 1e-2, 1e-3, 1e-4, 1e-5]
    lr_list = schedule.copy()
    lr_list.append(epoch)
    lr_list.sort()
    idx = lr_list.index(epoch)
    state['lr'] *= lr_set[idx]
    for param_group in optimizer.param_groups:
        param_group['lr'] = state['lr']

In [ ]:
for epoch in range(start_epoch, epochs):
    state['lr'] = optimizer.state_dict()['param_groups'][0]['lr']
    adjust_learning_rate(optimizer, epoch)
    print('\nEpoch: [%d | %d] LR: %f' % (epoch + 1, epochs, state['lr']))
    
    train_loss, train_acc = train(train_loader, model, criterion, optimizer, epoch, use_cuda)
    
    if epoch % 50 == 0:
        test_loss, test_acc = test(val_target_loader, model, criterion, epoch, use_cuda)
        source_loss, source_acc = test(val_source_loader, model, criterion, epoch, use_cuda)

        logger.append([state['lr'], train_loss, test_loss, train_acc, test_acc, source_loss, source_acc])
        scheduler_warmup.step()

        is_best = test_acc > best_acc
        best_acc = max(test_acc, best_acc)
        save_checkpoint({
            'epoch': epoch + 1,
            'state_dict' : model.state_dict(),
            'acc': test_acc,
            'best_acc': best_acc,
            'optimizer': optimizer.state_dict(),
        }, is_best, checkpoint=checkpoint)


Epoch: [1 | 5000] LR: 0.010000
1/13 | Total:0:00:01 | ETA:0:00:18 | Loss:3.5981335639953613 | top1:56.25
11/13 | Total:0:00:03 | ETA:0:00:01 | Loss:1.2446533278985457 | top1:64.20454406738281
13/13 | Total:0:00:03 | ETA:0:00:01 | Loss:1.1990931928157806 | top1:64.58333587646484


/home/cutz/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:21: UserWarning: volatile was removed and now has no effect. Use `with torch.no_grad():` instead.


107/107 | Total:0:00:21 | ETA:0:00:00 | Loss:1.0874167496912948 | top1:32.23987579345703
26/26 | Total:0:00:14 | ETA:0:00:00 | Loss:0.28156589086239153 | top1:86.1025619506836

Epoch: [2 | 5000] LR: 0.017000
1/13 | Total:0:00:00 | ETA:0:00:09 | Loss:0.5472944974899292 | top1:56.25
11/13 | Total:0:00:02 | ETA:0:00:01 | Loss:0.5676379935307936 | top1:68.18182373046875
13/13 | Total:0:00:02 | ETA:0:00:01 | Loss:0.5744919156034788 | top1:67.70833587646484

Epoch: [3 | 5000] LR: 0.017000
1/13 | Total:0:00:00 | ETA:0:00:08 | Loss:0.5314719676971436 | top1:68.75
11/13 | Total:0:00:02 | ETA:0:00:01 | Loss:0.5014441853219812 | top1:69.03409576416016
13/13 | Total:0:00:03 | ETA:0:00:01 | Loss:0.5100526586174965 | top1:68.75

Epoch: [4 | 5000] LR: 0.017000
1/13 | Total:0:00:00 | ETA:0:00:09 | Loss:0.5041465163230896 | top1:75.0
11/13 | Total:0:00:02 | ETA:0:00:01 | Loss:0.4893574795939706 | top1:73.57954406738281
13/13 | Total:0:00:03 | ETA:0:00:01 | Loss:0.48273494839668274 | top1:73.95833587646

1/13 | Total:0:00:00 | ETA:0:00:12 | Loss:0.23532332479953766 | top1:90.625
11/13 | Total:0:00:02 | ETA:0:00:01 | Loss:0.28200752626765857 | top1:85.2272720336914
13/13 | Total:0:00:03 | ETA:0:00:01 | Loss:0.2686486542224884 | top1:86.19792175292969

Epoch: [32 | 5000] LR: 0.017000
1/13 | Total:0:00:00 | ETA:0:00:10 | Loss:0.24608218669891357 | top1:81.25
11/13 | Total:0:00:02 | ETA:0:00:01 | Loss:0.2536258873614398 | top1:87.78409576416016
13/13 | Total:0:00:02 | ETA:0:00:01 | Loss:0.25747238968809444 | top1:88.02083587646484

Epoch: [33 | 5000] LR: 0.017000
1/13 | Total:0:00:00 | ETA:0:00:11 | Loss:0.22314661741256714 | top1:90.625
11/13 | Total:0:00:02 | ETA:0:00:01 | Loss:0.2707484540614215 | top1:87.21591186523438
13/13 | Total:0:00:03 | ETA:0:00:01 | Loss:0.27094163621465367 | top1:87.23958587646484

Epoch: [34 | 5000] LR: 0.017000
1/13 | Total:0:00:00 | ETA:0:00:08 | Loss:0.3082164525985718 | top1:93.75
11/13 | Total:0:00:02 | ETA:0:00:01 | Loss:0.24033401906490326 | top1:88.352

1/13 | Total:0:00:00 | ETA:0:00:10 | Loss:0.2843286991119385 | top1:87.5
11/13 | Total:0:00:02 | ETA:0:00:01 | Loss:0.20991349491206082 | top1:91.19318389892578
13/13 | Total:0:00:03 | ETA:0:00:01 | Loss:0.21666779120763144 | top1:90.88542175292969

Epoch: [61 | 5000] LR: 0.024000
1/13 | Total:0:00:00 | ETA:0:00:11 | Loss:0.3114086389541626 | top1:84.375
11/13 | Total:0:00:02 | ETA:0:00:01 | Loss:0.219112598083236 | top1:90.625
13/13 | Total:0:00:03 | ETA:0:00:01 | Loss:0.22008605177203813 | top1:90.625

Epoch: [62 | 5000] LR: 0.024000
1/13 | Total:0:00:00 | ETA:0:00:10 | Loss:0.13465118408203125 | top1:90.625
11/13 | Total:0:00:02 | ETA:0:00:01 | Loss:0.1793758516961878 | top1:91.76136779785156
13/13 | Total:0:00:03 | ETA:0:00:01 | Loss:0.17379577457904816 | top1:92.1875

Epoch: [63 | 5000] LR: 0.024000
1/13 | Total:0:00:00 | ETA:0:00:11 | Loss:0.12634438276290894 | top1:96.875
11/13 | Total:0:00:03 | ETA:0:00:01 | Loss:0.16106883436441422 | top1:92.8977279663086
13/13 | Total:0:00:03

...

1/13 | Total:0:00:00 | ETA:0:00:11 | Loss:0.011886926367878914 | top1:100.0
11/13 | Total:0:00:03 | ETA:0:00:01 | Loss:0.006402498788454316 | top1:100.0
13/13 | Total:0:00:03 | ETA:0:00:01 | Loss:0.0058882480176786585 | top1:100.0

Epoch: [1071 | 5000] LR: 0.079999
1/13 | Total:0:00:01 | ETA:0:00:15 | Loss:0.04319073259830475 | top1:96.875
11/13 | Total:0:00:03 | ETA:0:00:01 | Loss:0.08424190804362297 | top1:97.15909576416016
13/13 | Total:0:00:03 | ETA:0:00:01 | Loss:0.07946521447350581 | top1:97.39583587646484

Epoch: [1072 | 5000] LR: 0.079999
1/13 | Total:0:00:01 | ETA:0:00:14 | Loss:0.05508784204721451 | top1:96.875
11/13 | Total:0:00:03 | ETA:0:00:01 | Loss:0.017649426209655674 | top1:99.43182373046875
13/13 | Total:0:00:04 | ETA:0:00:01 | Loss:0.016532376563797396 | top1:99.47917175292969

Epoch: [1073 | 5000] LR: 0.079999
1/13 | Total:0:00:01 | ETA:0:00:16 | Loss:0.0008733645081520081 | top1:100.0
11/13 | Total:0:00:03 | ETA:0:00:01 | Loss:0.03328497792509469 | top1:98.57955169

1/13 | Total:0:00:01 | ETA:0:00:16 | Loss:0.06849557161331177 | top1:96.875
11/13 | Total:0:00:04 | ETA:0:00:01 | Loss:0.0453962538052689 | top1:98.01136779785156
13/13 | Total:0:00:04 | ETA:0:00:01 | Loss:0.04173727706074715 | top1:98.17708587646484

Epoch: [1100 | 5000] LR: 0.079999
1/13 | Total:0:00:01 | ETA:0:00:15 | Loss:0.08533063530921936 | top1:96.875
11/13 | Total:0:00:03 | ETA:0:00:01 | Loss:0.026866405016996643 | top1:99.1477279663086
13/13 | Total:0:00:03 | ETA:0:00:01 | Loss:0.02480204791451494 | top1:99.21875

Epoch: [1101 | 5000] LR: 0.079999
1/13 | Total:0:00:01 | ETA:0:00:15 | Loss:0.16694402694702148 | top1:93.75
11/13 | Total:0:00:03 | ETA:0:00:01 | Loss:0.02393788421018557 | top1:99.43182373046875
13/13 | Total:0:00:04 | ETA:0:00:01 | Loss:0.022160287170360487 | top1:99.47917175292969
107/107 | Total:0:00:24 | ETA:0:00:00 | Loss:0.5872988842915152 | top1:84.57009887695312
26/26 | Total:0:00:19 | ETA:0:00:00 | Loss:0.3770337872780286 | top1:88.5128173828125

Epoch: [

11/13 | Total:0:00:04 | ETA:0:00:01 | Loss:0.027749311686916786 | top1:99.1477279663086
13/13 | Total:0:00:04 | ETA:0:00:01 | Loss:0.02595202795540293 | top1:99.21875

Epoch: [1128 | 5000] LR: 0.079999
1/13 | Total:0:00:01 | ETA:0:00:19 | Loss:0.06012319028377533 | top1:96.875
11/13 | Total:0:00:04 | ETA:0:00:01 | Loss:0.011327533559365706 | top1:99.71591186523438
13/13 | Total:0:00:04 | ETA:0:00:01 | Loss:0.010793978658815226 | top1:99.73958587646484

Epoch: [1129 | 5000] LR: 0.079999
1/13 | Total:0:00:00 | ETA:0:00:12 | Loss:0.001316223293542862 | top1:100.0
11/13 | Total:0:00:02 | ETA:0:00:01 | Loss:0.03191552565179088 | top1:99.43182373046875
13/13 | Total:0:00:03 | ETA:0:00:01 | Loss:0.02929684069628517 | top1:99.47917175292969

Epoch: [1130 | 5000] LR: 0.079999
1/13 | Total:0:00:00 | ETA:0:00:09 | Loss:0.007441207766532898 | top1:100.0
11/13 | Total:0:00:03 | ETA:0:00:01 | Loss:0.0322889101437547 | top1:98.01136779785156
13/13 | Total:0:00:03 | ETA:0:00:01 | Loss:0.02969842761134

13/13 | Total:0:00:04 | ETA:0:00:01 | Loss:0.02491143314788739 | top1:99.21875

Epoch: [1156 | 5000] LR: 0.079999
1/13 | Total:0:00:01 | ETA:0:00:15 | Loss:0.0296830702573061 | top1:100.0
11/13 | Total:0:00:03 | ETA:0:00:01 | Loss:0.01735339957204732 | top1:99.43182373046875
13/13 | Total:0:00:03 | ETA:0:00:01 | Loss:0.018841494495670002 | top1:99.21875

Epoch: [1157 | 5000] LR: 0.079999
1/13 | Total:0:00:01 | ETA:0:00:13 | Loss:0.061340171843767166 | top1:96.875
11/13 | Total:0:00:03 | ETA:0:00:01 | Loss:0.0150693321431225 | top1:99.43182373046875
13/13 | Total:0:00:03 | ETA:0:00:01 | Loss:0.014141106201956669 | top1:99.47917175292969

Epoch: [1158 | 5000] LR: 0.079999
1/13 | Total:0:00:00 | ETA:0:00:10 | Loss:0.0014971643686294556 | top1:100.0
11/13 | Total:0:00:03 | ETA:0:00:01 | Loss:0.017925497313792057 | top1:99.1477279663086
13/13 | Total:0:00:03 | ETA:0:00:01 | Loss:0.016926166756699484 | top1:99.21875

Epoch: [1159 | 5000] LR: 0.079999
1/13 | Total:0:00:00 | ETA:0:00:11 | Loss

1/13 | Total:0:00:01 | ETA:0:00:13 | Loss:0.0007738173007965088 | top1:100.0
11/13 | Total:0:00:03 | ETA:0:00:01 | Loss:0.05391366559673439 | top1:98.29545593261719
13/13 | Total:0:00:03 | ETA:0:00:01 | Loss:0.050961569572488465 | top1:98.4375

Epoch: [1186 | 5000] LR: 0.079999
1/13 | Total:0:00:01 | ETA:0:00:19 | Loss:0.001719355583190918 | top1:100.0
11/13 | Total:0:00:04 | ETA:0:00:01 | Loss:0.02069893496280367 | top1:99.1477279663086
13/13 | Total:0:00:04 | ETA:0:00:01 | Loss:0.02067789885525902 | top1:99.21875

Epoch: [1187 | 5000] LR: 0.079999
1/13 | Total:0:00:01 | ETA:0:00:14 | Loss:0.08456730097532272 | top1:96.875
11/13 | Total:0:00:03 | ETA:0:00:01 | Loss:0.019884812222285705 | top1:99.43182373046875
13/13 | Total:0:00:04 | ETA:0:00:01 | Loss:0.018296433923145134 | top1:99.47917175292969

Epoch: [1188 | 5000] LR: 0.079999
1/13 | Total:0:00:01 | ETA:0:00:14 | Loss:0.046794816851615906 | top1:96.875
11/13 | Total:0:00:03 | ETA:0:00:01 | Loss:0.006902883506634019 | top1:99.7159